In [1]:
import pandas as pd 
import numpy as np
import os 
import seaborn as sns
import matplotlib as plt 
import glob
import random
import matplotlib.pyplot as plt
import re

In [2]:
# Cargar todos los archivos

# Directorio donde se encuentran los archivos archivos
folder_path = 'datos_estaciones'

# Obtener la lista de nombres de archivos en el directorio
file_list = os.listdir(folder_path)

# Realizar filtrado de todos los archivos del 2019, 2020, 2021, 2022, 2023 con extensión '.xls' 
filtered_files = [
    file for file in file_list
    if file.startswith(('Aemet2019','Aemet2020','Aemet2021','Aemet2022','Aemet2023')) and file.endswith('.xls')
]


# Crear una lista para almacenar los DataFrames de los archivos XLS
data_frames = []

# Función para limpiar y dividir la fecha
def clean_and_split_date(date_str):
    # Remove 'Aemet' from the date string
    cleaned_date = date_str.replace('Aemet', '')
    # Split the date into three columns: 'Año', 'Mes', 'Día'
    return cleaned_date.split('-')

# Leer iterativamente cada archivo para despues cargarlo como un DataFrame
for file in filtered_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, engine='xlrd')
    
    # Obtener el nombre del archivo sin la extensión
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Agregar el nombre del archivo a una columna nueva llamada 'Fecha'
    df['Fecha'] = file_name
    
    # Limpiar y dividir la columna 'Fecha'
    df[['Año', 'Mes', 'Día']] = df['Fecha'].apply(clean_and_split_date).apply(pd.Series)
    
    # Eliminar la columna 'Fecha'
    df.drop(columns=['Fecha'], inplace=True)
    
    data_frames.append(df)

# Concatenar los DataFrames en uno solo
combined_df = pd.concat(data_frames, ignore_index=True)

# Imprimir DataFrame
combined_df.head()

ValueError: No objects to concatenate

In [ ]:
# Limpiamos errores de carga del dataset

# Extraer las últimas tres columnas
ultimas_tres_columnas = combined_df.iloc[:, -3:]

# Eliminar las últimas siete columnas del DataFrame original
combined_df = combined_df.iloc[:, :-7]

# Eliminar las cuatro primeras filas de las últimas tres columnas
ultimas_tres_columnas = ultimas_tres_columnas.iloc[4:]

# Quitar las tres primeras filas
combined_df = combined_df.iloc[3:]

# Tomar la cuarta fila como nuevo encabezado
nuevo_encabezado = combined_df.iloc[:1]

# Eliminar la cuarta fila, que ahora es el nuevo encabezado
combined_df = combined_df.iloc[1:]

# Asignar el nuevo encabezado al DataFrame
combined_df.columns = nuevo_encabezado.iloc[0]

# Concatenar los DataFrames por columnas (axis=1)
combined_df = pd.concat([combined_df, ultimas_tres_columnas], axis=1)

# Restablecer el índice si es necesario
combined_df = combined_df.reset_index(drop=True)

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Eliminamos los paréntesis y el contenido dentro de ellos. 

def eliminar_parentesis(cadena):
    # Verificamos si el valor es un string antes de aplicar la función
    if isinstance(cadena, str):
        # Utilizamos una expresión regular para buscar el contenido entre paréntesis y lo eliminamos
        resultado = re.sub(r'\([^)]*\)', '', cadena)
        return resultado.strip()
    else:
        return cadena

# Aplicar la función a todo el DataFrame
combined_df = combined_df.applymap(eliminar_parentesis)

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Movemos las columnas para ordenar el dataset

# Obtener el número de columnas a mover desde el final del DataFrame
n_columnas_a_mover = 3

# Reorganizar el DataFrame moviendo las últimas n_columnas_a_mover al principio
combined_df = combined_df.iloc[:, -n_columnas_a_mover:].join(combined_df.iloc[:, :-n_columnas_a_mover])

# Nombre de la columna que deseas mover al principio
columna_a_mover = 'Estación'

# Obtener el índice de la columna a mover
indice_columna = combined_df.columns.get_loc(columna_a_mover)

# Reorganizar el DataFrame moviendo la columna al principio
combined_df = pd.concat([combined_df[columna_a_mover], combined_df.drop(columna_a_mover, axis=1)], axis=1)

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Del DataFrame solo nos interesan los datos de Barcelona. Eliminamos los que no necesitamos

# Eliminar las filas que contengan NaN en la columna 'provincia'
combined_df = combined_df.dropna(subset=['Provincia'])

# Filtrar las filas donde la columna 'provincia' contiene la palabra 'Barcelona'
combined_df = combined_df[combined_df['Provincia'].str.contains('Barcelona', case=False)]

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Rellenamos los NaN con la media de los 3 datos anteriores o en su defecto con el dato anterior

# Convertir las columnas a partir de la sexta al formato float
columnas_a_transformar = combined_df.columns[5:]
combined_df[columnas_a_transformar] = combined_df[columnas_a_transformar].astype(float)

# Función para rellenar los NaN con la media de los tres valores anteriores o duplicar el dato anterior
def rellenar_con_media_3_anteriores_o_duplicar(columna):
    columna_rolling = columna.rolling(3, min_periods=1).mean()
    columna_duplicada = columna.fillna(method='ffill')
    return np.where(columna_rolling.isnull(), columna_duplicada, columna_rolling)

# Aplicar la función a cada columna del DataFrame a partir de la sexta columna
for columna in combined_df.columns[5:]:
    combined_df[columna] = rellenar_con_media_3_anteriores_o_duplicar(combined_df[columna])

# Redondear los datos a dos decimales
combined_df = combined_df.round(2)

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Filtramos las estaciones que realmente necesitamos

# Filtrar los registros que contienen "Aeropuerto" o "marítimo" en la columna 'Estaciones'
combined_df = combined_df[combined_df['Estación'].str.contains('Aeropuerto|Marítimo')]

# Eliminar la columna 'Provincia'
combined_df = combined_df.drop(columns=['Provincia'])

# Eliminar las palabras 'Barcelona ' y 'Barcelona, ' de la columna 'Estación'
combined_df['Estación'] = combined_df['Estación'].str.replace('Barcelona, |Barcelona ', '', regex=True)

# Imprimir DataFrame
combined_df.head()

In [ ]:
# Guardar el DataFrame en un archivo Excel
combined_df.to_excel('Dataset_temperatura.xlsx', index=False)